In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os

In [3]:
embedding=nn.Embedding(5,10)
embedding(torch.tensor([0,0,1,1]))



tensor([[ 0.5825,  0.7791,  1.0392,  0.8744,  2.4429,  1.8319, -0.1873, -1.0729,
         -1.3501, -1.8906],
        [ 0.5825,  0.7791,  1.0392,  0.8744,  2.4429,  1.8319, -0.1873, -1.0729,
         -1.3501, -1.8906],
        [ 0.9430, -0.3195, -0.6365,  0.5277, -0.8741,  1.2561,  0.0276,  0.3325,
          0.3599,  0.9477],
        [ 0.9430, -0.3195, -0.6365,  0.5277, -0.8741,  1.2561,  0.0276,  0.3325,
          0.3599,  0.9477]], grad_fn=<EmbeddingBackward0>)

In [4]:
num_class=10
embed_len=20
latent_dim=100
img_size=28*28
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.label_embedding=nn.Embedding(num_class,embed_len)
        self.model=nn.Sequential(
            nn.Linear(latent_dim+embed_len, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.Linear(512, img_size),
            nn.Tanh()
            
        )
    def forward(self, z, label):
        # print(label.shape)
        c=self.label_embedding(label)
        print(c.shape)
        print(z.shape)
        x=torch.cat([z,c],dim=1)
        print(x.shape)
        return self.model(x)
        
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_Gen=Generator().to(device)
model_Gen



Generator(
  (label_embedding): Embedding(10, 20)
  (model): Sequential(
    (0): Linear(in_features=120, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=256, out_features=512, bias=True)
    (5): Linear(in_features=512, out_features=784, bias=True)
    (6): Tanh()
  )
)

In [5]:
batch=32
z=torch.randn(batch, latent_dim)
z.shape
z=torch.tensor(z)
label=torch.randint(1,num_class , (batch,))
label=torch.tensor(label)

model_Gen(z.to(device), label.to(device))

torch.Size([32, 20])
torch.Size([32, 100])
torch.Size([32, 120])


/tmp/ipykernel_13/1855804749.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  z=torch.tensor(z)
/tmp/ipykernel_13/1855804749.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label=torch.tensor(label)


tensor([[ 0.0082, -0.0518,  0.0612,  ...,  0.0834,  0.1702,  0.1147],
        [-0.0361, -0.0629,  0.1009,  ...,  0.1406, -0.0341,  0.1150],
        [ 0.0023, -0.0721,  0.0384,  ...,  0.0866, -0.0122,  0.0854],
        ...,
        [-0.0131, -0.0034,  0.0772,  ...,  0.0785, -0.0126,  0.0815],
        [ 0.1361, -0.0255,  0.0099,  ...,  0.0038, -0.0451,  0.1729],
        [ 0.0490,  0.0054,  0.0538,  ...,  0.0758,  0.0363,  0.0607]],
       grad_fn=<TanhBackward0>)